# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc038a4f550>
├── 'a' --> tensor([[ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012]])
└── 'x' --> <FastTreeValue 0x7fc038a4fee0>
    └── 'c' --> tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                        [ 0.4153, -1.1374,  0.7077,  0.0931],
                        [ 0.8926, -0.4442,  1.1661,  1.9294]])

In [4]:
t.a

tensor([[ 1.6751,  0.4012, -0.0407],
        [ 0.2470,  0.5189,  0.4012]])

In [5]:
%timeit t.a

60.7 ns ± 0.0139 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc038a4f550>
├── 'a' --> tensor([[ 0.7474,  0.1465, -1.0112],
│                   [-0.2000,  0.6134, -0.6970]])
└── 'x' --> <FastTreeValue 0x7fc038a4fee0>
    └── 'c' --> tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                        [ 0.4153, -1.1374,  0.7077,  0.0931],
                        [ 0.8926, -0.4442,  1.1661,  1.9294]])

In [7]:
%timeit t.a = new_value

65.1 ns ± 0.0265 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012]]),
    x: Batch(
           c: tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                      [ 0.4153, -1.1374,  0.7077,  0.0931],
                      [ 0.8926, -0.4442,  1.1661,  1.9294]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6751,  0.4012, -0.0407],
        [ 0.2470,  0.5189,  0.4012]])

In [11]:
%timeit b.a

56.9 ns ± 0.017 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6782, -0.0247, -2.1168],
               [-0.7172,  0.4886,  1.1220]]),
    x: Batch(
           c: tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                      [ 0.4153, -1.1374,  0.7077,  0.0931],
                      [ 0.8926, -0.4442,  1.1661,  1.9294]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 0.225 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc00ff767f0>
├── 'a' --> tensor([[[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]],
│           
│                   [[ 1.6751,  0.4012, -0.0407],
│                    [ 0.2470,  0.5189,  0.4012]]])
└── 'x' --> <FastTreeValue 0x7fbf8bc42b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 54.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc00ff76be0>
├── 'a' --> tensor([[ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012],
│                   [ 1.6751,  0.4012, -0.0407],
│                   [ 0.2470,  0.5189,  0.4012]])
└── 'x' --> <FastTreeValue 0x7fbf8b6b41f0>
    └── 'c' --> tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                        [ 0.4153, -1.1374,  0.7077,  0.0931],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 62.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.2 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]],
       
               [[ 1.6751,  0.4012, -0.0407],
                [ 0.2470,  0.5189,  0.4012]]]),
    x: Batch(
           c: tensor([[[-0.4580,  0.3277, -0.1731, -0.7808],
                       [ 0.4153, -1.1374,  0.7077,  0.0931],
                       [ 0.8926, -0.4442,  1.1661,  1.9294]],
         

In [26]:
%timeit Batch.stack(batches)

79.8 µs ± 95.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012],
               [ 1.6751,  0.4012, -0.0407],
               [ 0.2470,  0.5189,  0.4012]]),
    x: Batch(
           c: tensor([[-0.4580,  0.3277, -0.1731, -0.7808],
                      [ 0.4153, -1.1374,  0.7077,  0.0931],
                      [ 0.8926, -0.4442,  1.1661,  1.9294],
                      [-0.4580,  0.3277, -0.1731, -0.7808],
                      [ 0.4153, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 6.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
